# Image features exercise
*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

We have seen that we can achieve reasonable performance on an image classification task by training a linear classifier on the pixels of the input image. In this exercise we will show that we can improve our classification performance by training linear classifiers not on raw pixels but on features that are computed from the raw pixels.

All of your work for this exercise will be done in this notebook.

In [ ]:
from __future__ import print_function

import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('../../')

from datasets import CiFarDataSet as DataSet

%matplotlib inline
plt.rcParams['figure.figsize'] = (10., 10.)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk.

In [54]:
from features import color_histogram_hsv, hog_feature


cifar = DataSet(input_height=32,
                input_width=32,
                input_channel=3,
                split_rate=0.02,
                name='cifar-10')  # loading CiFar-10 DataSet

x_train = cifar.train_images
y_train = cifar.train_labels
x_val = cifar.valid_images
y_val = cifar.valid_labels
x_test = cifar.test_images
y_test = cifar.test_labels

# print the shapes
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(49000, 32, 32, 3) (49000, 10)
(1000, 32, 32, 3) (1000, 10)
(10000, 32, 32, 3) (10000, 10)


## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram using the hue channel in HSV
color space. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone. Verifying this assumption would
be a good thing to try for the bonus section.

The `hog_feature` and `color_histogram_hsv` functions both operate on a single
image and return a feature vector for that image. The extract_features
function takes a set of images and a list of feature functions and evaluates
each feature function on each image, storing the results in a matrix where
each column is the concatenation of all feature vectors for a single image.

In [55]:
from features import *


num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, n_bin=num_color_bins)]
x_train_feats = extract_features(x_train, feature_fns)
x_val_feats = extract_features(x_val, feature_fns)
x_test_feats = extract_features(x_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(x_train_feats, axis=0, keepdims=True)
x_train_feats -= mean_feat
x_val_feats -= mean_feat
x_test_feats -= mean_feat

# Preprocessing: Divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(x_train_feats, axis=0, keepdims=True)
x_train_feats /= std_feat
x_val_feats /= std_feat
x_test_feats /= std_feat

# Preprocessing: Add a bias dimension
x_train_feats = np.hstack([x_train_feats, np.ones((x_train_feats.shape[0], 1))])
x_val_feats = np.hstack([x_val_feats, np.ones((x_val_feats.shape[0], 1))])
x_test_feats = np.hstack([x_test_feats, np.ones((x_test_feats.shape[0], 1))])

Done extracting features for 2000 / 49000 images
Done extracting features for 4000 / 49000 images
Done extracting features for 6000 / 49000 images
Done extracting features for 8000 / 49000 images
Done extracting features for 10000 / 49000 images
Done extracting features for 12000 / 49000 images
Done extracting features for 14000 / 49000 images
Done extracting features for 16000 / 49000 images
Done extracting features for 18000 / 49000 images
Done extracting features for 20000 / 49000 images
Done extracting features for 22000 / 49000 images
Done extracting features for 24000 / 49000 images
Done extracting features for 26000 / 49000 images
Done extracting features for 28000 / 49000 images
Done extracting features for 30000 / 49000 images
Done extracting features for 32000 / 49000 images
Done extracting features for 34000 / 49000 images
Done extracting features for 36000 / 49000 images
Done extracting features for 38000 / 49000 images
Done extracting features for 40000 / 49000 images
Done

## Train SVM on features
Using the multiclass SVM code developed earlier in the assignment, train SVMs on top of the features extracted above; this should achieve better results than training SVMs directly on top of raw pixels.

In [73]:
# Use the validation set to tune the learning rate and regularization strength

from classifiers.linear_classifier import LinearSVM


learning_rates = [1e-9, 1e-8, 1e-7]
regularization_strengths = [5e4, 5e5, 5e6]

results = {}
best_val = -1
best_svm = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained classifer in best_svm. You might also want to play          #
# with different numbers of bins in the color histogram. If you are careful    #
# you should be able to get accuracy of near 0.44 on the validation set.       #
################################################################################

for lr in learning_rates:
    for reg in regularization_strengths:
        # Linear SVM by lr & reg
        svm = LinearSVM(batch_size=200, learning_rate=lr, regularization=reg)
        
        # training
        svm.train(x_train_feats, y_train)
        
        # prediction
        y_train_pred = svm.predict(x_train_feats)
        y_val_pred = svm.predict(x_val_feats)
        
        # accuracy
        train_acc = np.mean(y_train_pred == y_train)
        val_acc = np.mean(y_val_pred == y_val)
        
        # acc results
        results[(lr, reg)] = (train_acc, val_acc)
        
        # update best acc & svm
        if val_acc > best_val:
            best_val = val_acc
            best_svm = svm

################################################################################
#                              END OF YOUR CODE                                #
################################################################################

# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (lr, reg, train_accuracy, val_accuracy))

print('best validation accuracy achieved during cross-validation: %f' % best_val)

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
# Evaluate your trained SVM on the test set
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)

print(test_accuracy)

In [ ]:
# An important way to gain intuition about how an algorithm works is to
# visualize the mistakes that it makes. In this visualization, we show examples
# of images that are misclassified by our current system. The first column
# shows images that our system labeled as "plane" but whose true label is
# something other than "plane".

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)

    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')

        if i == 0:
            plt.title(cls_name)

plt.show()

### Inline question 1:
Describe the misclassification results that you see. Do they make sense?

Answer : well.... maybe :)...

## Neural Network on image features
Earlier in this assigment we saw that training a two-layer neural network on raw pixels achieved better classification performance than linear classifiers on raw pixels. In this notebook we have seen that linear classifiers on image features outperform linear classifiers on raw pixels. 

For completeness, we should also try training a neural network on image features. This approach should outperform all previous approaches: you should easily be able to achieve over 55% classification accuracy on the test set; our best model achieves about 60% classification accuracy.

In [ ]:
print(X_train_feats.shape)

In [ ]:
from classifiers.neural_net import TwoLayerNet

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net = None

################################################################################
# TODO: Train a two-layer neural network on image features. You may want to    #
# cross-validate various parameters as in previous sections. Store your best   #
# model in the best_net variable.                                              #
################################################################################
pass
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

In [ ]:
# Run your neural net classifier on the test set. You should be able to
# get more than 55% accuracy.

test_acc = np.mean(net.predict(X_test_feats) == y_test)

print(test_acc)

# Bonus: Design your own features!

You have seen that simple image features can improve classification performance. So far we have tried HOG and color histograms, but other types of features may be able to achieve even better classification performance.

For bonus points, design and implement a new type of feature and use it for image classification on CIFAR-10. Explain how your feature works and why you expect it to be useful for image classification. Implement it in this notebook, cross-validate any hyperparameters, and compare its performance to the HOG + Color histogram baseline.

# Bonus: Do something extra!
Use the material and code we have presented in this assignment to do something interesting. Was there another question we should have asked? Did any cool ideas pop into your head as you were working on the assignment? This is your chance to show off!